In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap

In [ ]:
activity = {}

In [ ]:
activity['2020'] = pd.read_excel('./2020.xlsx', skiprows=6, sheet_name="Transaction Details")
activity['2021'] = pd.read_excel('./2021.xlsx', skiprows=6, sheet_name="Transaction Details")
activity['2022.1'] = pd.read_excel('./2022-pt1.xlsx', skiprows=6, sheet_name="Transaction Details")


In [ ]:
activities = pd.concat(activity.values())
activities = activities[activities.Amount > -200]

In [ ]:
activities[activities.Amount < 0].sample(10)

In [ ]:
activities.head(10).Description.str.extract(r'^((?:\S+\s+){2}\S+).*').groupby(by=0).count()

In [ ]:
activity.Description.str.extract(r'^((?:\S+\s+){2}\S+).*', expand=False).str.lower()

In [ ]:
def apply_filter(row, filter_string):
    return row['Amount'] if  filter_string in str(row['trunc_desc']) else 0

In [ ]:
def apply_group_filter(row, group_list):
    return any(filter_string in str(row['trunc_desc']) for filter_string in group_list)

In [ ]:
grocers = []


In [ ]:
recurring = []

In [ ]:
april_set = set(enriched[enriched.tran_month=='2022-04-01'].trunc_desc.values)

In [ ]:
feb_set = set(enriched[enriched.tran_month=='2022-02-01'].trunc_desc.values)

In [ ]:
march_set = set(enriched[enriched.tran_month=='2022-03-01'].trunc_desc.values)

In [ ]:
recent_set =  [x for x in april_set.intersection(march_set) if str(x) != 'nan'] 

In [ ]:
recent_set

In [ ]:
set(recent_set).intersection(set(recurring))

In [ ]:
set(recent_set).difference(set(recurring))

In [ ]:
def enrich_df(df):
    df['trunc_desc'] = (df.Description
                        .str.extract(r'^((?:\S+\s+){2}\S+).*', expand=False)
                        .str.lower()
                       )
    df['tran_month'] = pd.to_datetime(df.Date).dt.to_period('M').dt.to_timestamp()
    df['recurring']= df.apply(lambda x: apply_group_filter(x, recurring), axis=1)
    df['recent_recurring']= df.apply(lambda x: apply_group_filter(x, recent_set), axis=1)
    return df

In [ ]:
enriched = enrich_df(activities)
enriched_recurring = enriched[enriched.recurring]
enriched_recent = enriched[enriched.recent_recurring]

In [ ]:
enriched.duplicated()

In [ ]:
enriched_recurring

In [ ]:
enriched.groupby(['tran_month']).sum('Amount').reset_index(level=[0])

In [ ]:
(ggplot(enriched.groupby(['tran_month']).sum('Amount').reset_index(level=[0]), aes('tran_month', 'Amount'), )
 + geom_point())
#  + stat_smooth(method='lm'))
#  + facet_wrap('~gear'))

In [ ]:
(ggplot(enriched_recurring.groupby(['tran_month']).sum('Amount').reset_index(level=[0]), aes('tran_month', 'Amount'), )
 + geom_point())
#  + stat_smooth(method='lm'))
#  + facet_wrap('~gear'))

In [ ]:
(ggplot(enriched_recent.groupby(['tran_month']).sum('Amount').reset_index(level=[0]), aes('tran_month', 'Amount'), )
 + geom_point())
#  + stat_smooth(method='lm'))
#  + facet_wrap('~gear'))

In [ ]:
per_month = enriched.groupby(['trunc_desc', 'tran_month', 'Category']).sum().reset_index(level=[0,1,2])

In [ ]:
enriched.groupby(['Category']).sum('Amount').reset_index(level=[0]).sort_values(by='Amount', ascending= False)

In [ ]:
enriched_recurring.groupby(['Category']).sum('Amount').reset_index(level=[0]).sort_values(by='Amount', ascending= False)

In [ ]:
(enriched[enriched.Category=='Merchandise & Supplies-Hardware Supplies']
.groupby(['trunc_desc'])
 .sum()
 .sort_values(by='Amount', ascending= False)
)

In [ ]:
(enriched[enriched.Category=='Merchandise & Supplies-Internet Purchase']
.groupby(['trunc_desc'])
 .sum()
 .sort_values(by='Amount', ascending= False)
)

In [ ]:
(enriched[enriched.Category=='Merchandise & Supplies-Groceries']
.groupby(['trunc_desc'])
 .sum()
 .sort_values(by='Amount', ascending= False)
)

In [ ]:
(enriched[enriched.Category=='Business Services-Contracting Services']
.groupby(['trunc_desc'])
 .sum()
 .sort_values(by='Amount', ascending= False)
)

In [ ]:
(enriched[enriched.Category=='Business Services-Health Care Services']
.groupby(['trunc_desc'])
 .sum()
 .sort_values(by='Amount', ascending= False)
)

In [ ]:
(enriched[enriched.Category=='Merchandise & Supplies-General Retail']
.groupby(['trunc_desc'])
 .sum()
 .sort_values(by='Amount', ascending= False)
)

In [ ]:
(enriched[enriched.Category=='Merchandise & Supplies-Computer Supplies']
.groupby(['trunc_desc'])
 .sum()
 .sort_values(by='Amount', ascending= False)
)

In [ ]:
plotnine.options.figure_size = (6.5, 12)

In [ ]:
(ggplot(enriched.groupby(['tran_month', 'Category']).sum('Amount').reset_index(level=[0,1]), 
        aes('tran_month', 'Amount', color='Category'))
 + geom_point())
#  + stat_smooth(method='lm'))
#  + facet_wrap('~Category'))

In [ ]:
(ggplot(enriched_recent.groupby(['tran_month', 'Category']).sum('Amount').reset_index(level=[0,1]), 
        aes('tran_month', 'Amount', color='Category'))
 + geom_point())
#  + stat_smooth(method='lm'))
#  + facet_wrap('~Category'))

In [ ]:
(ggplot(enriched_recurring.groupby(['tran_month', 'Category']).sum('Amount').reset_index(level=[0,1]), 
        aes('tran_month', 'Amount', color='Category'))
 + geom_point())
#  + stat_smooth(method='lm'))
#  + facet_wrap('~Category'))

In [ ]:
sums = per_month.groupby(['tran_month', 'Category']).sum().reset_index(level=[0,1])

In [ ]:
sums[sums.Category == 'Merchandise & Supplies-Groceries']

In [ ]:
vendor_sums = per_month.groupby(['trunc_desc', 'tran_month']).sum().reset_index(level=[0,1])

In [ ]:
vendor_sums

In [ ]:
enriched['recurring']= enriched.apply(lambda x: apply_group_filter(x, recurring), axis=1)

In [ ]:
def calc_group(row, group__list):
    return row.filter(items=grocers).sum()

In [ ]:
activity.filter(items=grocers).sum()

In [ ]:
filtered = enriched_recent[enriched_recent.trunc_desc != 'the home depot']

In [ ]:
filtered.groupby('trunc_desc').sum().sort_values(by='Amount', ascending=False)

In [ ]:
(ggplot(filtered.groupby(['tran_month', 'trunc_desc']).sum('Amount').reset_index(level=[0,1]), 
        aes('tran_month', 'Amount', color='trunc_desc'))
 + geom_point())
#  + stat_smooth(method='lm'))
#  + facet_wrap('~Category'))

In [ ]:
(ggplot(filtered.groupby(['tran_month']).sum('Amount').reset_index(level=[0]), 
        aes('tran_month', 'Amount'))
 + geom_point())
#  + stat_smooth(method='lm'))
#  + facet_wrap('~Category'))

In [ ]:
april_set = set(enriched[enriched.tran_month=='2022-04-01'].trunc_desc.values)

In [ ]:
may_set = set(enriched[enriched.tran_month=='2022-05-01'].trunc_desc.values)

In [ ]:
march_set = set(enriched[enriched.tran_month=='2022-03-01'].trunc_desc.values)

In [ ]:
april_set.union(may_set).union(march_set)